# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 3 2022 11:15AM,238733,15,VT80173228,"Virtus Pharmaceuticals, LLC",Released
1,Jun 3 2022 11:15AM,238733,15,VT80173229,"Virtus Pharmaceuticals, LLC",Released
2,Jun 3 2022 11:15AM,238733,15,VT80173230,"Virtus Pharmaceuticals, LLC",Released
3,Jun 3 2022 11:15AM,238733,15,VT80173231,"Virtus Pharmaceuticals, LLC",Released
4,Jun 3 2022 11:15AM,238733,15,VT80173232,"Virtus Pharmaceuticals, LLC",Released
5,Jun 3 2022 11:15AM,238733,15,VT80173233,"Virtus Pharmaceuticals, LLC",Released
6,Jun 3 2022 11:15AM,238733,15,VT80173234,"Virtus Pharmaceuticals, LLC",Released
7,Jun 3 2022 11:15AM,238733,15,VT80173235,"Virtus Pharmaceuticals, LLC",Released
8,Jun 3 2022 11:15AM,238733,15,VT80173236,"Virtus Pharmaceuticals, LLC",Released
9,Jun 3 2022 11:15AM,238733,15,VT80173237,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,238748,Released,1
37,238749,Released,10
38,238751,Released,1
39,238752,Released,1
40,238753,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238748,NaN,NaN,1.0
238749,NaN,NaN,10.0
238751,NaN,NaN,1.0
238752,NaN,NaN,1.0
238753,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238650,0.0,0.0,1.0
238654,0.0,0.0,1.0
238662,0.0,0.0,2.0
238670,3.0,6.0,3.0
238682,6.0,13.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238650,0,0,1
238654,0,0,1
238662,0,0,2
238670,3,6,3
238682,6,13,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238650,0,0,1
1,238654,0,0,1
2,238662,0,0,2
3,238670,3,6,3
4,238682,6,13,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238650,,,1
1,238654,,,1
2,238662,,,2
3,238670,3,6,3
4,238682,6,13,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 3 2022 11:15AM,238733,15,"Virtus Pharmaceuticals, LLC"
18,Jun 3 2022 11:10AM,238753,10,Bio-PRF
19,Jun 3 2022 11:10AM,238752,10,Bio-PRF
20,Jun 3 2022 11:09AM,238751,10,Beach Patient Assistance
21,Jun 3 2022 11:05AM,238749,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
31,Jun 3 2022 10:48AM,238748,16,Sartorius Stedim Filters Inc.
32,Jun 3 2022 10:42AM,238747,10,ISDIN Corporation
33,Jun 3 2022 10:22AM,238746,10,Eywa Pharma Inc.
40,Jun 3 2022 10:13AM,238745,16,Sartorius Bioprocess Solutions
41,Jun 3 2022 10:12AM,238744,10,"Amcyte Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 3 2022 11:15AM,238733,15,"Virtus Pharmaceuticals, LLC",,,18
1,Jun 3 2022 11:10AM,238753,10,Bio-PRF,,,1
2,Jun 3 2022 11:10AM,238752,10,Bio-PRF,,,1
3,Jun 3 2022 11:09AM,238751,10,Beach Patient Assistance,,,1
4,Jun 3 2022 11:05AM,238749,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,10
5,Jun 3 2022 10:48AM,238748,16,Sartorius Stedim Filters Inc.,,,1
6,Jun 3 2022 10:42AM,238747,10,ISDIN Corporation,,,1
7,Jun 3 2022 10:22AM,238746,10,Eywa Pharma Inc.,,,7
8,Jun 3 2022 10:13AM,238745,16,Sartorius Bioprocess Solutions,,,1
9,Jun 3 2022 10:12AM,238744,10,"Amcyte Pharma, Inc.",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 11:15AM,238733,15,"Virtus Pharmaceuticals, LLC",18,,
1,Jun 3 2022 11:10AM,238753,10,Bio-PRF,1,,
2,Jun 3 2022 11:10AM,238752,10,Bio-PRF,1,,
3,Jun 3 2022 11:09AM,238751,10,Beach Patient Assistance,1,,
4,Jun 3 2022 11:05AM,238749,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,10,,
5,Jun 3 2022 10:48AM,238748,16,Sartorius Stedim Filters Inc.,1,,
6,Jun 3 2022 10:42AM,238747,10,ISDIN Corporation,1,,
7,Jun 3 2022 10:22AM,238746,10,Eywa Pharma Inc.,7,,
8,Jun 3 2022 10:13AM,238745,16,Sartorius Bioprocess Solutions,1,,
9,Jun 3 2022 10:12AM,238744,10,"Amcyte Pharma, Inc.",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 11:15AM,238733,15,"Virtus Pharmaceuticals, LLC",18,,
1,Jun 3 2022 11:10AM,238753,10,Bio-PRF,1,,
2,Jun 3 2022 11:10AM,238752,10,Bio-PRF,1,,
3,Jun 3 2022 11:09AM,238751,10,Beach Patient Assistance,1,,
4,Jun 3 2022 11:05AM,238749,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,10,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 11:15AM,238733,15,"Virtus Pharmaceuticals, LLC",18.0,NaN,NaN
1,Jun 3 2022 11:10AM,238753,10,Bio-PRF,1.0,NaN,NaN
2,Jun 3 2022 11:10AM,238752,10,Bio-PRF,1.0,NaN,NaN
3,Jun 3 2022 11:09AM,238751,10,Beach Patient Assistance,1.0,NaN,NaN
4,Jun 3 2022 11:05AM,238749,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,10.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 11:15AM,238733,15,"Virtus Pharmaceuticals, LLC",18.0,0.0,0.0
1,Jun 3 2022 11:10AM,238753,10,Bio-PRF,1.0,0.0,0.0
2,Jun 3 2022 11:10AM,238752,10,Bio-PRF,1.0,0.0,0.0
3,Jun 3 2022 11:09AM,238751,10,Beach Patient Assistance,1.0,0.0,0.0
4,Jun 3 2022 11:05AM,238749,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,10.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4058365,103.0,23.0,4.0
12,716176,24.0,2.0,0.0
15,238733,18.0,0.0,0.0
16,716147,3.0,0.0,0.0
18,477441,2.0,0.0,0.0
19,716117,12.0,1.0,0.0
20,716145,14.0,19.0,6.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4058365,103.0,23.0,4.0
1,12,716176,24.0,2.0,0.0
2,15,238733,18.0,0.0,0.0
3,16,716147,3.0,0.0,0.0
4,18,477441,2.0,0.0,0.0
5,19,716117,12.0,1.0,0.0
6,20,716145,14.0,19.0,6.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,103.0,23.0,4.0
1,12,24.0,2.0,0.0
2,15,18.0,0.0,0.0
3,16,3.0,0.0,0.0
4,18,2.0,0.0,0.0
5,19,12.0,1.0,0.0
6,20,14.0,19.0,6.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,103.0
1,12,Released,24.0
2,15,Released,18.0
3,16,Released,3.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20
Status,,,,,,,
Completed,4.0,0.0,0.0,0.0,0.0,0.0,6.0
Executing,23.0,2.0,0.0,0.0,0.0,1.0,19.0
Released,103.0,24.0,18.0,3.0,2.0,12.0,14.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0,6.0
1,Executing,23.0,2.0,0.0,0.0,0.0,1.0,19.0
2,Released,103.0,24.0,18.0,3.0,2.0,12.0,14.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0,6.0
1,Executing,23.0,2.0,0.0,0.0,0.0,1.0,19.0
2,Released,103.0,24.0,18.0,3.0,2.0,12.0,14.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()